# Puzzle

https://thefiddler.substack.com/p/bingo

### This Week’s Fiddler

A game of bingo typically consists of a 5-by-5 grid with 25 total squares. Each square (except for the center square) contains a number. When a square’s number is called, you place a marker on that square. The goal is to get “bingo,” which is five squares in a row, either across, down, or along one of the two long diagonals. The center square, which doesn’t have a number, is labeled “Free,” and begins with a marker on it before any numbers are called. Here’s an example of a winning 5-by-5 grid in which 10 squares (other than the “Free” square) have been marked:

Consider a smaller version of the game with a 3-by-3 grid: a “Free” square surrounded by eight other squares with numbers. Suppose each of these eight squares is equally likely to be called, and without replacement (i.e., once a number is called, it doesn’t get called again).

On average, how many markers must you place until you get “bingo” in this 3-by-3 grid? (The “Free” square doesn’t count as one of the markers—it’s “free”.)

### This Week’s Extra Credit

Instead of a 3-by-3 grid, let’s return to the original 5-by-5 grid.

On average, how many markers must you place until you get “bingo”? (As before, the “Free” square doesn’t count as one of the markers—it’s “free”.)

# Solution

I think it makes most sense to write some code here.

In [90]:
from fractions import Fraction as F
def expected_markers_before_bingo(n, cells_placed=[], debug=False):
    """
    Calculate the expected number of markers drawn before achieving bingo
    in a simplified bingo game with n unique markers.

    Parameters:
    n (int): The total number of unique rows and columns in the bingo grid.
    markers (list): The list of markers already drawn. Each value is 0 to n-1.

    Returns:
    float: The expected number of additional markers to draw before achieving bingo.
    """
    num_cells_placed = len(cells_placed)
    if num_cells_placed >= n:
        # Check rows
        for r in range(n):
            if all(r * n + c in cells_placed for c in range(n)):
                return 0
        # Check columns
        for c in range(n):
            if all(r * n + c in cells_placed for r in range(n)):
                return 0
        # Check main diagonal
        if all(r * n + r in cells_placed for r in range(n)):
            return 0
        # Check anti-diagonal
        if all(r * n + (n - 1 - r) in cells_placed for r in range(n)):
            return 0

    exp_val_terms = []        
    num_cells_to_check = n * n if (num_cells_placed > 1) else (n * n) // 2 # Use symmetry for first pick
    for marker in range(num_cells_to_check):
        if marker not in cells_placed:
            new_cells_placed = cells_placed + [marker]
            exp_val_terms.append(1 + expected_markers_before_bingo(n, new_cells_placed, debug))
    
    if not exp_val_terms:        
        exp_val = -1000000  # Should not happen
    else:
        exp_val = F(sum(exp_val_terms) , len(exp_val_terms))

    if debug:
        if (num_cells_placed <= 3):  # Limit output to initial states
            print(f"Expected markers for n={n} with cells_placed={cells_placed}: {exp_val} = Avg [{exp_val_terms}]")
    return exp_val

assert(expected_markers_before_bingo(2, []) == 2)

In [91]:
def solve_puzzle(n):
    middle_cell = (n // 2) * n + (n // 2)
    return expected_markers_before_bingo(n, [middle_cell], debug=False)

In [92]:
Fiddler_Answer = solve_puzzle(3)
print("Fiddler Answer:", Fiddler_Answer, "=", float(Fiddler_Answer))

Fiddler Answer: 243/70 = 3.4714285714285715


Let's check the fiddler answer.

We need to place at least 2 cells. 

I think at 5 cells placed you are sure to have a bingo. This is because each placed cell has an anti-cell across the free cell, and at 4 cells placed you have 4 anti-cells as well, and placing any of them will make a match.
e.g.
| | | |
|-|-|-|
|P|X|X|
|X|F|P|
|P|P|X|

So, the answer should be something between 2 and 5, i.e. some wieghted average of 2,3,4,5.

For bingo in 2 cells placed, the probability is 1/7. First cell can be anything, and the next cell has to be the corresponding anti-cell (1 out of 7).

For bingo in 3 cells placed, the probability is 6/7 * (2/6 + 12/28 * 1/6) [ (not 2 case) * (completion using the free cell + completion not using the free cell ) ]. This is 1/7 * (2 + 3/7) = 17/49.

| Steps to Bingo | P | Cumulative P|
|-|-|-|
| 2 | 1/7 | 1/7 |
| 3 | 17/49 | 24/49 |
| 4 | ? | ? |
| 5 | ? | 1 |

So, something around 3.5 is very plausible.

In [93]:
#Extra_Credit_Answer = solve_puzzle(5)
#print("Extra Credit Answer:", Extra_Credit_Answer)

# This cannot complete in reasonable time for n=5 due to combinatorial explosion.

The extra credit version cannot complete in an hour or more, so let me try to write something faster using bitmasks instead of lists, and using caching to speed things up.

In [94]:
from functools import cache
@cache
def calc_bingo_masks(n):
    masks = []
    for r in range(n):
        row_mask = 0
        col_mask = 0
        for c in range(n):
            row_mask |= (1 << (r * n + c))
            col_mask |= (1 << (c * n + r))
        masks.append(row_mask)
        masks.append(col_mask)
    diag1_mask = 0
    diag2_mask = 0
    for r in range(n):
        diag1_mask |= (1 << (r * n + r))
        diag2_mask |= (1 << (r * n + (n - 1 - r)))
    masks.append(diag1_mask)
    masks.append(diag2_mask)
    return masks

@cache
def calc_top_left_mask(n):
    mask = 0
    for r in range((n-1)//2):
        for c in range((n+1)//2):
             mask |= 1 << (r * n + c)
    return mask

In [95]:

@cache
def expected_markers_before_bingo_using_mask(n, cells_placed_mask):
    """
    Calculate the expected number of markers drawn before achieving bingo
    in a simplified bingo game with n unique markers.

    Parameters:
    n (int): The total number of unique rows and columns in the bingo grid.
    cells_placed_mask (int): A bitmask representing the cells already drawn. Each bit corresponds to a cell in the n x n grid.

    Returns:
    float: The expected number of additional markers to draw before achieving bingo.
    """
    num_cells_placed = cells_placed_mask.bit_count()
    if num_cells_placed >= n:
        bingo_masks = calc_bingo_masks(n)
        for mask in bingo_masks:
            if (cells_placed_mask & mask) == mask:
                # Shout BINGO
                return 0
    
    exp_val_sum, exp_val_count = 0, 0  
    top_left_mask = calc_top_left_mask(n)
    for marker in range(n * n):
        marker_bit = 1 << marker
        if not (cells_placed_mask & marker_bit):            
            if (num_cells_placed != 1 or (marker_bit & top_left_mask) != 0): # Use 4 way symmetry for first pick 
                new_cells_placed_mask = cells_placed_mask | marker_bit
                exp_val_sum += 1 + expected_markers_before_bingo_using_mask(n, new_cells_placed_mask)
                exp_val_count += 1
    
    exp_val = F(exp_val_sum, exp_val_count) if exp_val_count > 0 else -1000000  # Should not happen

    if False:  #debug
        if (num_cells_placed <= 3):  # Limit output to initial states
            print(f"Expected markers for n={n} with cells_placed_mask={cells_placed_mask}: {exp_val} = Avg [{exp_val_terms}]")
    return exp_val

#assert(expected_markers_before_bingo_using_mask(2, 0) == 2)

def solve_puzzle_using_mask(n):
    middle_cell = (n // 2) * n + (n // 2)
    middle_cell_mask = 1 << middle_cell
    return expected_markers_before_bingo_using_mask(n, middle_cell_mask)

In [96]:
Fiddler_Answer_using_mask = solve_puzzle_using_mask(3)
print("Fiddler Answer using mask:", Fiddler_Answer_using_mask, "=", float(Fiddler_Answer_using_mask))

Fiddler Answer using mask: 243/70 = 3.4714285714285715


In [97]:
Extra_Credit_Answer = solve_puzzle_using_mask(5)
print("Extra Credit Answer:", Extra_Credit_Answer, "=", float(Extra_Credit_Answer))

Extra Credit Answer: 4245967/312018 = 13.608083508002743


Here the range of possible values is 4 to 21, I think, so something in the middle, i.e. close to 12.5 would be expected. 13.6 sounds pretty plausible.

# Conclusion

Fiddler Answer = 243/70 = 3.4714285714285715

Extra Credit Answer = 4245967/312018 = 13.608083508002743